In [ ]:
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from skimage import io, transform
from PIL import Image
import os
import numpy as np

In [ ]:
from torch.utils.data import DataLoader
import torch

In [ ]:
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os

In [ ]:
label_dic={'Animal Husbandry':0,'Certificates':1,'Community Infrastructure':2,'Crime and Safety':3,'Garbage and Unsanitary Practices':9,'Mobility,Roads,Footpath,Infrustructure':10,
           'Lakes':4,'Others':5,'Parks & Recreation':6,'Pollution':7,'Sewerage Systems':8,'Storm Water Drains':11,'Street Lighting':12,'Traffic and Road Safety':13,'Trees and Saplings':14,'Water Supply and Services':15}

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path='/content/drive/My Drive/traindata/', transforms=None):
        self.path = path
        l=os.listdir(path)
        l = os.listdir(self.path)
        st=path
        for ind,ele in enumerate(l):
          l[ind]=st+ele
        self.list=[]
        l1=[os.path.join(l[0],i) for i in os.listdir(l[0])]
        l2=[os.path.join(l[1],i) for i in os.listdir(l[1])]
        l3=[os.path.join(l[2],i) for i in os.listdir(l[2])]
        self.list.extend(l1)
        self.list.extend(l2)
        self.list.extend(l3)
        self.transforms = transforms
        
    def __getitem__(self, index):
        # stuff
        # print(self.list[index].split('/')[-1].split('(')[0])
        image_label = label_dic[self.list[index].split('/')[-2].strip()]
        image = Image.open(self.list[index])
        # print(image_label)
        # print('image')
        image = image.convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)
        return (image, image_label)
    def __len__(self):
        return len(self.list)

In [ ]:
transformations_train=transforms.Compose([
        transforms.Resize((227,227)),
        transforms.RandomAffine(degrees=[15,15], 
                                translate=(0.05, 0.2), 
                                ),transforms.ToTensor()
        
    ])

In [ ]:
transformations_test = transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor()])

In [ ]:
train_set = CustomDataset(transforms = transformations_train)

In [ ]:
batch_size=256

In [ ]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle = True)

In [ ]:
test_set = CustomDataset(path='/content/drive/My Drive/testdata/',transforms = transformations_test)

In [ ]:
len(train_loader.dataset)

3488

In [ ]:
len(test_loader.dataset)

282

In [ ]:
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle = True)

In [ ]:
alexnet = models.alexnet(pretrained=True)

In [ ]:
# alexnet

In [ ]:
alexnet.classifier[6] = nn.Linear(4096, 3) # assuming that the fc7 layer has 512 neurons, otherwise change it 
alexnet=alexnet.cuda()

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
summary(alexnet,(3,227,227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
device='cuda'

In [ ]:
def train_model(model, dataloaders, criterion, optimizer):
  
      model.train()
      running_loss = 0.0
      running_corrects = 0
       
      for inputs, labels in dataloaders:
          inputs = inputs.to(device)
          
          labels = labels.to(device)

          
          optimizer.zero_grad()

          
          with torch.set_grad_enabled(True):
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              _, preds = torch.max(outputs, 1)

              
              loss.backward()
              optimizer.step()

          
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / len(dataloaders.dataset)
      epoch_acc = running_corrects.double() / len(dataloaders.dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format('train', epoch_loss, epoch_acc))

      torch.save(model.state_dict(), '/content/drive/My Drive/Alexnet_weights.pth.tar')
      return epoch_loss,epoch_acc


In [ ]:
def test_model(model, dataloaders, criterion, optimizer):
      model.eval()
      running_loss = 0.0
      running_corrects = 0
       
      for inputs, labels in dataloaders:
          inputs = inputs.to(device)
          
          labels = labels.to(device)          
          
          with torch.set_grad_enabled(False):
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              _, preds = torch.max(outputs, 1)

         
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / len(dataloaders.dataset)
      epoch_acc = running_corrects.double() / len(dataloaders.dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format('test', epoch_loss, epoch_acc))
      return epoch_loss,epoch_acc

In [ ]:
optimizer = torch.optim.SGD(squeezenet.parameters(), lr=0.0003,momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()


In [ ]:
import pickle

In [ ]:
test_loader

In [ ]:
num_epochs=20
train_acc=[]
train_loss=[]
test_acc=[]
test_loss=[]
for epoch in range(num_epochs):
  print('Epoch {}/{}'.format(epoch, num_epochs - 1))
  print('-' * 10)
  if epoch>16:
      print(epoch)
      for g in optimizer.param_groups:
        g['lr'] = 0.001
  epoch_loss,epoch_acc=train_model(alexnet,train_loader, criterion, optimizer)
  train_acc.append(epoch_acc)
  train_loss.append(epoch_loss)
  print(epoch)
  epoch_loss1,epoch_acc1=test_model(alexnet,test_loader, criterion, optimizer)
  test_acc.append(epoch_acc1)
  test_loss.append(epoch_loss1)
  f=open('/content/drive/My Drive/sml1_train_acc.pkl', 'wb') 
  pickle.dump(train_acc, f)
  f.close()
  f=open('/content/drive/My Drive/sml1_train_loss.pkl', 'wb') 
  pickle.dump(train_loss, f)
  f.close()
  f=open('/content/drive/My Drive/sml1_test_acc.pkl', 'wb') 
  pickle.dump(test_acc, f)
  f.close()
  f=open('/content/drive/My Drive/sml1_test_loss.pkl', 'wb') 
  pickle.dump(train_acc, f)
  f.close()

Epoch 0/19
----------
train Loss: 28.5221 Acc: 0.3363
0
test Loss: 1.0953 Acc: 0.3498
Epoch 1/19
----------


/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


train Loss: 1.2885 Acc: 0.3653
1
test Loss: 1.1619 Acc: 0.3598
Epoch 2/19
----------
train Loss: 1.1139 Acc: 0.3627
2
test Loss: 1.1023 Acc: 0.3535
Epoch 3/19
----------
train Loss: 1.1001 Acc: 0.3607
3
test Loss: 1.0988 Acc: 0.3598
Epoch 4/19
----------


KeyboardInterrupt: ignored